In [1]:
# Import packages
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor
from IPython.display import clear_output

# Set working path
path = '/Users/martinbogaert/Desktop/NBA Data Analysis/nba_ai_awards/'

print('MAKE SURE YOU CHANGE THE WEEK VARIABLE')

MAKE SURE YOU CHANGE THE WEEK VARIABLE


In [2]:
week = 17 # CHANGE THIS
date = datetime.now().strftime('%d %B %Y')
time = datetime.now().strftime('%H:%M:%S')
print(date + ' ; ' + time)

13 February 2023 ; 18:08:51


In [3]:
features = ['PTS','VORP','FG','WS','MP','TRB','AST','PER','BPM']#, 'GS'] ### GS
model = RandomForestRegressor(n_estimators = 100, max_features = 'auto', min_samples_leaf = 2)

# Minimum minutes per game
mp = 8
# Propotion of maximum games played
gp = 1/4

In [4]:
# Load training data
train_data = pd.read_csv(path + 'Algorithm/roy/roy_data.csv')

# Load predicted data
data = pd.read_csv(path + f'Algorithm/weekly data/week_{week}.csv') # Load up-to-date data
data = data.groupby('roy').get_group(1)

# Load weekly info
with open(path + f'Algorithm/weekly data/week_{week}.txt', 'r') as file:
            date1 = file.readline().splitlines()[0][7:]
            date2 = file.readline().splitlines()[0][7:]
            n = int(file.readline()[12:])
            
season_progress = n / 15
data.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A,roy,smoy
0,A.J. Green,SG,23,MIL,25,0,8.8,1.5,3.4,0.440,...,3.02,114.35,111.43,2.92,2.82,114.33,111.58,2.74,1,1
11,MarJon Beauchamp,SF,22,MIL,37,9,14.6,2.2,5.4,0.404,...,3.02,114.35,111.43,2.92,2.82,114.33,111.58,2.74,1,1
17,A.J. Lawson,SG,22,DAL,12,0,4.2,0.8,1.7,0.450,...,0.71,117.19,116.48,0.71,0.83,117.07,116.26,0.82,1,1
25,Jaden Hardy,SG,20,DAL,29,0,11.2,2.2,5.2,0.430,...,0.71,117.19,116.48,0.71,0.83,117.07,116.26,0.82,1,1
36,AJ Griffin,SF,19,ATL,53,11,20.5,3.7,7.7,0.479,...,0.11,115.52,115.41,0.11,-0.10,115.55,115.66,-0.11,1,1


In [5]:
# Set training data
X_train = train_data[features]
y_train = train_data['Share']

# Set minimum requirements (games and minutes played)
data = data[data['MP'] >= mp] # Minimum of minutes per game
data = data[data['G'] > season_progress * gp] # Minimum games played

pred = []
for step in range(0, 10) :
    print('Model ' + str(step+1) + '/10 ...')
    clear_output(wait = True)
    # Fit Machine Learning model
    model.fit(X_train, y_train)

    # Predict shares of test data
    pred.append(model.predict(data[features]))

Model 10/10 ...


In [6]:
# Assemble results DataFrame
res = data.assign(Share = [np.array(pred)[:,i].mean() for i in range(0, len(data))])
res = res.sort_values('Share', ascending = False) # Sort values by predictions
res['Rank'] = list(range(1, len(res)+1))
res['week'] = len(res) * [week] # Add week

res = res.reset_index(drop = True)[['Player', 'MP' ,'Tm', 'PER', 'VORP', 'WS', 'FG', 'PTS', 'Seed', 'Share']]
res.head(5)

,Player,MP,Tm,PER,VORP,WS,FG,PTS,Seed,Share
0,Paolo Banchero,33.7,ORL,15.2,0.328000,2.788000,6.5,20.0,13,0.257665
1,Bennedict Mathurin,28.2,IND,13.5,-0.848276,2.262069,5.4,17.2,12,0.129243
2,Walker Kessler,21.1,UTA,21.3,1.938182,7.156364,3.6,8.2,11,0.127038
3,Jalen Duren,25.2,DET,17.2,0.656000,5.740000,3.7,8.8,14,0.093731
4,Jaden Ivey,30.1,DET,12.2,-0.759259,0.303704,5.1,15.2,14,0.068012


In [7]:
res.to_csv(path + f'Results/roy/results_week_{week}.csv', index = None)